In [1]:
import pandas as pd
import numpy as np
import nltk
nltk.download('punkt')
!pip install psycopg2
import psycopg2
import json

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\gr801\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
connection = psycopg2.connect(dbname='news_classifier', user='postgres', 
                        password='Vadim14112003', host='localhost')

In [3]:
def get(q):
    cursor = connection.cursor()
    cursor.execute(q)
    res = cursor.fetchall()
    cursor.close()
    return res

In [4]:
def split_on_sentences(s):
    return nltk.tokenize.sent_tokenize(s)

In [5]:
def sorting_by_key(d: dict):
    return dict(sorted(d.items(), key = lambda x: len(x[0]), reverse=True))

In [6]:
def find_keys(text):
    for i in sorted_tokens:
        text = text.replace(i, sorted_tokens[i])
    return text

In [7]:
def split_on_words(s):
    tokenizer = nltk.RegexpTokenizer(r'\w+')
    return tokenizer.tokenize(s)

In [8]:
def word_classification(t):
    if t in inv_tokens:
        t = inv_tokens[t]
        if t in game_tags:
            t = t.split()
            return [(t[k], "B-g") if k == 0
                    else (t[k], "I-g")
                    for k in range(len(t))
                   ]
        elif t in developers_tags:
            t = t.split()
            return [(t[k], "B-d") if k == 0
                    else (t[k], "I-d")
                    for k in range(len(t))
                   ]
    else:
        return [(t, "O")]

## Tokens ##

In [9]:
game_tags = list(pd.read_csv('all_games.csv')['name'].values) + ['Dungeons & Dragons', 'D&D']

In [10]:
len(game_tags)

18802

In [11]:
import json
with open('Developers.json', "r") as f:
    data = json.load(f)
    developers = pd.DataFrame(data)
developers_tags = developers.Developer.values

In [12]:
len(developers_tags)

721

In [13]:
tokens = list(game_tags) + list(developers_tags)
tokens = np.unique(tokens)

In [14]:
tokens = {tokens[i]: f"t_{i}" for i in np.arange(len(tokens))}
inv_tokens = {v: k for k, v in tokens.items()}

In [15]:
sorted_tokens = sorting_by_key(tokens)

# Articles ##

In [16]:
q = "SELECT text FROM public.gamespot_article_content"
sentences = get(q)
sentences = np.array([i[0] for i in sentences])

In [17]:
sentences = [s for i in sentences for s in split_on_sentences(i)]

In [18]:
sentences = np.array([i for i in sentences if any(t in i for t in tokens.keys())])

In [19]:
sentences

array(["Dungeons & Dragons executive producer announced that they're pulling back from the planned launch of Open Gaming License 1.2, which would replace and deauthorize Open Gaming License 1.0.",
       '"When you give us playtest feedback, we take it seriously," Brink wrote.',
       'Untouched," Brink continued.', ...,
       "Preorders are the reason why the game is experiencing a surge of popularity on Steam's store, and that theme was echoed by Destiny 2: The Witch Queen in seventh place.",
       'You can see the full results below from third-party site SteamDB: Steam Top Sellers February 14-20 Cyberpunk 2077--which is in fifth place--has seen a revival on the Steam Charts now that the game has received a substantial update.',
       'New DLC finally lets you snap photos of Johnny Silverhand, the frustratingly long braindance tutorial can be skipped, and more romantic content allows you to receive some saucy photos occasionally.'],
      dtype='<U20241')

In [20]:
sentences = np.array([find_keys(i) for i in sentences])

In [21]:
sentences

array(["t_3155 executive producer announced that they're pulling back from the planned launch of Open Gaming License 1.2, which would replace and deauthorize Open Gaming License 1.0.",
       '"When you give us playtest feedback, we take it seriously," t_1419 wrote.',
       'Untouched," t_1419 continued.', ...,
       "Preorders are the reason why the game is experiencing a surge of popularity on Steam's store, and that theme was echoed by t_2633: The Witch Queen in seventh place.",
       'You can see the full results below from third-party site SteamDB: Steam Top Sellers t_3772bruary 14-20 t_2245--which is in fifth place--has seen a revival on the Steam Charts now that the game has received a substantial update.',
       'New DLC finally lets you snap photos of Johnny Silverhand, the frustratingly long bt_12968dance tutorial can be skipped, and more romantic content allows you to receive some saucy photos occasionally.'],
      dtype='<U20364')

In [22]:
len(sentences)

64386

In [23]:
sentences = list(sentences)

In [27]:
sentences

["t_3155 executive producer announced that they're pulling back from the planned launch of Open Gaming License 1.2, which would replace and deauthorize Open Gaming License 1.0.",
 '"When you give us playtest feedback, we take it seriously," t_1419 wrote.',
 'Untouched," t_1419 continued.',
 't_1419 himself explains the importance of putting the SRD, or System Reference Document, under the Creative Commons license.',
 "The System Reference Document version 5.1 is the most recent version of the open-source t_2249 rules, which allows third parties to use t_3155' basic rules to create additional t_2249 content or build other games using the rule set.",
 'It\'s open and irrevocable in a way that doesn\'t require you to take [Wizards of the Coast\'s] word for it," t_1419 wrote.',
 'As t_1419 states, the company cannot take the SRD 5.1 document out of Creative Commons after putting it there, and they have no control over the wording of that license.',
 "The trouble with that move is that t_22

In [28]:
import json
with open("clean_sentences.json", 'w') as f:
    json.dump(sentences, f)

In [30]:
sentences = np.array([(i, w) for i in np.arange(len(sentences)) for w in split_on_words(sentences[i])])

In [31]:
sentences

array([['0', 't_3155'],
       ['0', 'executive'],
       ['0', 'producer'],
       ...,
       ['64385', 'saucy'],
       ['64385', 'photos'],
       ['64385', 'occasionally']], dtype='<U93')

In [33]:
sentences = np.array([(i[0], w, t) for i in sentences for w, t in word_classification(i[1])])

In [34]:
sentences

array([['0', 'Dungeons', 'B-g'],
       ['0', '&', 'I-g'],
       ['0', 'Dragons', 'I-g'],
       ...,
       ['64385', 'saucy', 'O'],
       ['64385', 'photos', 'O'],
       ['64385', 'occasionally', 'O']], dtype='<U93')

In [36]:
sentences = pd.DataFrame(sentences, columns=["sentence", "word", "tag"])

In [37]:
sentences

,sentence,word,tag
0,0,Dungeons,B-g
1,0,&,I-g
2,0,Dragons,I-g
3,0,executive,O
4,0,producer,O
...,...,...,...
2148744,64385,receive,O
2148745,64385,some,O
2148746,64385,saucy,O
2148747,64385,photos,O


In [39]:
len(np.unique(sentences['word']))

58484

In [38]:
sentences.to_csv("gamespot_ner.csv")

In [40]:
sentences

,sentence,word,tag
0,0,Dungeons,B-g
1,0,&,I-g
2,0,Dragons,I-g
3,0,executive,O
4,0,producer,O
...,...,...,...
2148744,64385,receive,O
2148745,64385,some,O
2148746,64385,saucy,O
2148747,64385,photos,O


In [44]:
len(sentences[sentences['tag'] == "O"])

1999659

In [45]:
len(sentences)

2148749

In [47]:
1999659/2148749 -1 

-0.06938455817780487